# Q2 Supervised Learning

In [1]:
# Import the necessary libraries
import pandas as pd
import numpy as np

from project_1.config import PROJ_ROOT, PROCESSED_DATA_DIR
from project_1.loading import *

SEED = 42
np.random.seed(SEED)

In [2]:
# Load the data using the functions from the loading module

# Load final data without ICU (not used in training)
set_a, set_b, set_c = load_final_data_without_ICU()
death_a, death_b, death_c = load_outcomes()

Shapes of the datasets:
Set A: (183416, 42) Set B: (183495, 42) Set C: (183711, 42)
Shapes of labels:
Set A: (4000, 2) Set B: (4000, 2) Set C: (4000, 2)


In [3]:
# Check if there are any missing values in the data
print(f"Missing values in set_a: {set_a.isnull().sum().sum()}")
print(f"Missing values in set_b: {set_b.isnull().sum().sum()}")
print(f"Missing values in set_c: {set_c.isnull().sum().sum()}")
print(f"Missing values in death_a: {death_a.isnull().sum().sum()}")
print(f"Missing values in death_b: {death_b.isnull().sum().sum()}")
print(f"Missing values in death_c: {death_c.isnull().sum().sum()}")

Missing values in set_a: 0
Missing values in set_b: 0
Missing values in set_c: 0
Missing values in death_a: 0
Missing values in death_b: 0
Missing values in death_c: 0


In [7]:
# Check if the outcomes only contain 0s and 1s
print("Death A contains only 0 and 1:", death_a["In-hospital_death"].isin([0, 1]).all())
print("Death B contains only 0 and 1:", death_b["In-hospital_death"].isin([0, 1]).all())
print("Death C contains only 0 and 1:", death_c["In-hospital_death"].isin([0, 1]).all())

Death A contains only 0 and 1: True
Death B contains only 0 and 1: True
Death C contains only 0 and 1: True


***

## Q2.1 Classic Machine Learning Methods

### Part 1 - Train two distinct ML classifiers after aggregating rows

In [8]:
# Prepare the datasets

# Change RecordID to int32
set_a["RecordID"] = set_a["RecordID"].astype(np.int32)
set_b["RecordID"] = set_b["RecordID"].astype(np.int32)
set_c["RecordID"] = set_c["RecordID"].astype(np.int32)

# Include the outcomes in the datasets
set_a = set_a.merge(death_a, on="RecordID", how="left")
set_b = set_b.merge(death_b, on="RecordID", how="left")
set_c = set_c.merge(death_c, on="RecordID", how="left")

# Change names of the datasets
train_set = set_a
val_set = set_b
test_set = set_c

train_set.head()

,RecordID,Time,Gender,Height,Weight,Age,Albumin,Cholesterol,DiasABP,HCO3,...,WBC,pH,MechVent,TroponinT,ALP,ALT,AST,Bilirubin,TroponinI,In-hospital_death
0,132539,2025-03-10 00:00:00,0.0,-0.950526,-0.23008,-0.596332,1.671639,-0.013487,-0.832594,-0.109176,...,0.753623,1.125,0.0,1.923077,0.132075,-0.176471,0.450704,1.545455,0.285714,0
1,132539,2025-03-10 01:00:00,0.0,-0.950526,-0.23008,-0.596332,1.967793,0.172112,-0.608431,-0.109176,...,-0.420290,0.125,0.0,-0.246154,0.000000,-0.294118,0.436620,0.000000,-0.126984,0
2,132539,2025-03-10 02:00:00,0.0,-0.950526,-0.23008,-0.596332,-1.734132,0.125712,0.848629,0.830987,...,-0.014493,-0.875,0.0,0.000000,0.773585,-0.205882,-0.380282,0.181818,-0.095238,0
3,132539,2025-03-10 03:00:00,0.0,-0.950526,-0.23008,-0.596332,1.523562,0.380910,-0.832594,-0.579257,...,0.188406,-0.375,0.0,0.215385,-0.698113,-0.588235,1.126761,-0.181818,4.650794,0
4,132539,2025-03-10 04:00:00,0.0,-0.950526,-0.23008,-0.596332,0.487023,-0.964680,1.483758,-0.814297,...,-1.144928,1.000,0.0,2.738462,-0.490566,-0.558824,-0.225352,0.363636,0.904762,0


In [10]:
# Define aggregation rules
aggregation_rules = {
    "Age": "last",
    "Gender": "last",
    "Height": "last",
    "Albumin": "last",
    "ALP": "last",
    "ALT": "last",
    "AST": "last",
    "Bilirubin": "last",
    "BUN": "last",
    "Cholesterol": "last",
    "Creatinine": "last",
    "DiasABP": "mean",
    "FiO2": "mean",
    "GCS": "min",
    "Glucose": "mean",
    "HCO3": "last",
    "HCT": "last",
    "HR": "mean",
    "K": "last",
    "Lactate": "max",
    "Mg": "last",
    "MAP": "mean",
    "MechVent": "last",
    "Na": "last",
    "NIDiasABP": "mean",
    "NIMAP": "mean",
    "NISysABP": "mean",
    "PaCO2": "last",
    "PaO2": "mean",
    "pH": "last",
    "Platelets": "last",
    "RespRate": "mean",
    "SaO2": "mean",
    "SysABP": "mean",
    "Temp": "max",
    "TroponinI": "max",
    "TroponinT": "max",
    "Urine": "sum",
    "WBC": "last",
    "Weight": "last",
    "In-hospital_death": "max"  # If any 1 exists for a patient, return 1
}

# Perform aggregation
train_aggregated = train_set.groupby("RecordID").agg(aggregation_rules).reset_index()
val_aggregated = val_set.groupby("RecordID").agg(aggregation_rules).reset_index()
test_aggregated = test_set.groupby("RecordID").agg(aggregation_rules).reset_index()


# Display the processed dataset
print("Shape of the aggregated training set:", train_aggregated.shape)
print("Shape of the aggregated validation set:", val_aggregated.shape)
print("Shape of the aggregated test set:", test_aggregated.shape)
# Display the first few rows of the aggregated training set
train_aggregated.head()

Shape of the aggregated training set: (4000, 42)
Shape of the aggregated validation set: (4000, 42)
Shape of the aggregated test set: (4000, 42)


,RecordID,Age,Gender,Height,Albumin,ALP,ALT,AST,Bilirubin,BUN,...,RespRate,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,Weight,In-hospital_death
0,132539,-0.596332,0.0,-0.950526,-1.289901,-0.716981,-0.411765,0.802817,-0.181818,-0.588235,...,-0.302039,-0.427083,-0.108444,1.501788,6.095238,34.892308,43.142857,-0.318841,-0.230080,0
1,132540,0.667051,1.0,0.617613,-1.289901,-0.716981,-0.411765,0.802817,-0.181818,0.176471,...,-0.019156,-0.632653,0.001751,1.123672,6.095238,34.892308,37.022619,0.246377,-0.232705,0
2,132541,-1.170597,0.0,-0.429614,-1.141824,0.471698,1.529412,1.718310,1.909091,-0.882353,...,-0.018740,-1.166667,-0.053937,2.510098,6.095238,34.892308,30.971429,-0.782609,-1.077244,0
3,132543,0.207639,1.0,1.157978,1.967793,0.471698,-0.323529,-0.380282,-0.454545,-0.470588,...,-0.722062,-0.408163,-0.152114,-0.010676,6.095238,34.892308,135.657143,-0.536232,0.143617,0
4,132545,1.356169,0.0,-1.252050,0.338946,-0.716981,-0.411765,0.802817,-0.181818,0.411765,...,-0.023980,-0.414894,-0.184575,0.997634,6.095238,34.892308,7.114286,-0.985507,-0.809442,0


## Model 1 - Logistic Regression

In [11]:
# Separate Predictors (X) and Target (y)
X_train = train_aggregated.drop(columns=["RecordID", "In-hospital_death"])
y_train = train_aggregated["In-hospital_death"]

X_val = val_aggregated.drop(columns=["RecordID", "In-hospital_death"])
y_val = val_aggregated["In-hospital_death"]

X_test = test_aggregated.drop(columns=["RecordID", "In-hospital_death"])
y_test = test_aggregated["In-hospital_death"]

# Visualize the shape of the datasets
print(X_train.shape, y_train.shape)


(4000, 40) (4000,)


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score

# Separate features and target for each set.

# Create and train the Logistic Regression classifier.
clf = LogisticRegression(random_state=SEED, max_iter=1000)
clf.fit(X_train, y_train)

# Evaluate on the validation set
y_valid_proba = clf.predict_proba(X_val)[:, 1]  # probability for the positive class
roc_auc_valid = roc_auc_score(y_val, y_valid_proba)
auprc_valid = average_precision_score(y_val, y_valid_proba)
print(f"Validation ROC AUC: {roc_auc_valid:.3f}, AUPRC: {auprc_valid:.3f}")

# Evaluate on the test set.
y_test_proba = clf.predict_proba(X_test)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Validation ROC AUC: 0.790, AUPRC: 0.419
Test ROC AUC: 0.770, AUPRC: 0.392


## Model 2 - Random Forest

In [13]:
# Use Random Forest to predict the target
from sklearn.ensemble import RandomForestClassifier

# Create and train the Random Forest classifier.
clf = RandomForestClassifier(random_state=SEED)
clf.fit(X_train, y_train)

# Optionally, evaluate on the validation set.
y_valid_proba = clf.predict_proba(X_val)[:, 1]  # probability for the positive class
roc_auc_valid = roc_auc_score(y_val, y_valid_proba)
auprc_valid = average_precision_score(y_val, y_valid_proba)
print(f"Validation ROC AUC: {roc_auc_valid:.3f}, AUPRC: {auprc_valid:.3f}")

# Evaluate on the test set.
y_test_proba = clf.predict_proba(X_test)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Validation ROC AUC: 0.791, AUPRC: 0.431
Test ROC AUC: 0.778, AUPRC: 0.410


***

# Part 2 - Feature Engineering

We use Feature Lagging and also TSFresh for time-series relevant feature extraction

## Feature Lagging

In [14]:
# Investigate the best features to expand with lagged values
def compute_patientwise_avg_acf(df, feature, lag=1):
    """
    Compute the average lag-1 autocorrelation for a given feature across patients.
    """
    acf_values = []
    for rid, group in df.groupby("RecordID"):
        series = group[feature].dropna()
        if len(series) < lag + 1:
            continue
        acf_val = series.corr(series.shift(lag))
        if pd.notna(acf_val):
            acf_values.append(acf_val)
    if len(acf_values) > 0:
        return np.mean(acf_values)
    else:
        return None

#! Using train_set because it's before the aggregation
# Define the candidate features: exclude static ones ("RecordID", "Time", "In-hospital_death").
candidate_features = [col for col in train_set.columns if col not in ["RecordID", "Time", "In-hospital_death", "Age", "Weight", "Height", "Gender"]]

# Set threshold for absolute autocorrelation (lag 1)
threshold = 0.5
selected_features = []
for feature in candidate_features:
    avg_acf = compute_patientwise_avg_acf(train_set, feature, lag=1)
    if avg_acf is not None and abs(avg_acf) >= threshold:
        selected_features.append(feature)
        #print(f"Selected {feature} with average lag-1 ACF = {avg_acf:.3f}")

print("Features selected for lag augmentation:", selected_features)

/Users/francescobondi/anaconda3/envs/TUM/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/francescobondi/anaconda3/envs/TUM/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/Users/francescobondi/anaconda3/envs/TUM/lib/python3.12/site-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/francescobondi/anaconda3/envs/TUM/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Users/francescobondi/anaconda3/envs/TUM/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


Features selected for lag augmentation: ['HCO3', 'HCT', 'HR', 'Mg', 'Na', 'Temp', 'BUN', 'FiO2', 'GCS', 'K']


In [19]:
# Check the number of NaN values
print(train_set.isnull().sum().sum())

0


Now add the lag columns for those selected features.
WARNING! This operation creates NaN values inside the DataFrame

In [15]:
def add_lag_features_for_selected(df, selected_features, lags=[1,2]):
    """
    For each feature in selected_features, add lag features computed patient-wise.
    """
    df_augmented = df.copy()
    for feature in selected_features:
        if feature not in df_augmented.columns:
            continue
        for lag in lags:
            lag_col = f"{feature}_lag{lag}"
            df_augmented[lag_col] = df_augmented.groupby("RecordID")[feature].shift(lag)
    return df_augmented

# Augment train, validation, and test sets with lag features using the selected_features list.
train_set_aug = add_lag_features_for_selected(train_set, selected_features, lags=[1,2])
valid_set_aug = add_lag_features_for_selected(val_set, selected_features, lags=[1,2])
test_set_aug  = add_lag_features_for_selected(test_set,  selected_features, lags=[1,2])

# Print shapes
print(train_set_aug.shape, valid_set_aug.shape, test_set_aug.shape)

"""train_set_clean = train_set_aug.dropna()
valid_set_clean = valid_set_aug.dropna()
test_set_clean  = test_set_aug.dropna()

print(f"Train set shape after dropping NaNs: {train_set_clean.shape}")
print(f"Validation set shape after dropping NaNs: {valid_set_clean.shape}")
print(f"Test set shape after dropping NaNs: {test_set_clean.shape}")"""
train_set_aug.head()

(183416, 63) (183495, 63) (183711, 63)


,RecordID,Time,Gender,Height,Weight,Age,Albumin,Cholesterol,DiasABP,HCO3,...,Temp_lag1,Temp_lag2,BUN_lag1,BUN_lag2,FiO2_lag1,FiO2_lag2,GCS_lag1,GCS_lag2,K_lag1,K_lag2
0,132539,2025-03-10 00:00:00,0.0,-0.950526,-0.23008,-0.596332,1.671639,-0.013487,-0.832594,-0.109176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,132539,2025-03-10 01:00:00,0.0,-0.950526,-0.23008,-0.596332,1.967793,0.172112,-0.608431,-0.109176,...,-0.514830,NaN,-0.176471,NaN,2.5,NaN,0.166667,NaN,1.875,NaN
2,132539,2025-03-10 02:00:00,0.0,-0.950526,-0.23008,-0.596332,-1.734132,0.125712,0.848629,0.830987,...,-2.090314,-0.514830,-0.058824,-0.176471,1.5,2.5,0.166667,0.166667,0.125,1.875
3,132539,2025-03-10 03:00:00,0.0,-0.950526,-0.23008,-0.596332,1.523562,0.380910,-0.832594,-0.579257,...,-2.090314,-2.090314,0.117647,-0.058824,0.0,1.5,0.166667,0.166667,0.000,0.125
4,132539,2025-03-10 04:00:00,0.0,-0.950526,-0.23008,-0.596332,0.487023,-0.964680,1.483758,-0.814297,...,-2.090314,-2.090314,-0.588235,0.117647,0.0,0.0,0.166667,0.166667,0.625,0.000


In [16]:
# Print the number of NaN values after augmenting the features
print(train_set_aug.isnull().sum().sum(), valid_set_aug.isnull().sum().sum(), test_set_aug.isnull().sum().sum())

119970 119930 119980


## Perform Aggregation on the new augmented datasets

In [17]:
extended_aggregation_rules = aggregation_rules.copy()
for col in train_set_aug.columns:
    if "lag" in col:
        extended_aggregation_rules[col] = "last"

print("Extended aggregation rules:")
print(extended_aggregation_rules)

# Now perform aggregation on the train, validation, and test sets.
train_aggregated = train_set_aug.groupby("RecordID").agg(extended_aggregation_rules).reset_index()
val_aggregated   = valid_set_aug.groupby("RecordID").agg(extended_aggregation_rules).reset_index()
test_aggregated  = test_set_aug.groupby("RecordID").agg(extended_aggregation_rules).reset_index()

# Display the processed dataset (for example, for the train set)
print(train_aggregated.shape)
train_aggregated.head()

Extended aggregation rules:
{'Age': 'last', 'Gender': 'last', 'Height': 'last', 'Albumin': 'last', 'ALP': 'last', 'ALT': 'last', 'AST': 'last', 'Bilirubin': 'last', 'BUN': 'last', 'Cholesterol': 'last', 'Creatinine': 'last', 'DiasABP': 'mean', 'FiO2': 'mean', 'GCS': 'min', 'Glucose': 'mean', 'HCO3': 'last', 'HCT': 'last', 'HR': 'mean', 'K': 'last', 'Lactate': 'max', 'Mg': 'last', 'MAP': 'mean', 'MechVent': 'last', 'Na': 'last', 'NIDiasABP': 'mean', 'NIMAP': 'mean', 'NISysABP': 'mean', 'PaCO2': 'last', 'PaO2': 'mean', 'pH': 'last', 'Platelets': 'last', 'RespRate': 'mean', 'SaO2': 'mean', 'SysABP': 'mean', 'Temp': 'max', 'TroponinI': 'max', 'TroponinT': 'max', 'Urine': 'sum', 'WBC': 'last', 'Weight': 'last', 'In-hospital_death': 'max', 'HCO3_lag1': 'last', 'HCO3_lag2': 'last', 'HCT_lag1': 'last', 'HCT_lag2': 'last', 'HR_lag1': 'last', 'HR_lag2': 'last', 'Mg_lag1': 'last', 'Mg_lag2': 'last', 'Na_lag1': 'last', 'Na_lag2': 'last', 'Temp_lag1': 'last', 'Temp_lag2': 'last', 'BUN_lag1': 'last'

,RecordID,Age,Gender,Height,Albumin,ALP,ALT,AST,Bilirubin,BUN,...,Temp_lag1,Temp_lag2,BUN_lag1,BUN_lag2,FiO2_lag1,FiO2_lag2,GCS_lag1,GCS_lag2,K_lag1,K_lag2
0,132539,-0.596332,0.0,-0.950526,-1.289901,-0.716981,-0.411765,0.802817,-0.181818,-0.588235,...,0.745556,0.745556,-0.588235,-0.588235,0.0,-0.5,0.166667,0.166667,-0.125,-0.125
1,132540,0.667051,1.0,0.617613,-1.289901,-0.716981,-0.411765,0.802817,-0.181818,0.176471,...,-0.136714,-0.136714,0.176471,0.176471,-0.5,-0.5,0.166667,0.166667,-0.750,-0.750
2,132541,-1.170597,0.0,-0.429614,-1.141824,0.471698,1.529412,1.718310,1.909091,-0.882353,...,0.241402,1.375750,-0.882353,-0.882353,-0.5,-0.5,-1.500000,-1.500000,-0.500,-0.500
3,132543,0.207639,1.0,1.157978,1.967793,0.471698,-0.323529,-0.380282,-0.454545,-0.470588,...,-1.775217,-1.775217,-0.470588,-0.470588,0.0,-0.5,0.166667,0.166667,-0.375,-0.375
4,132545,1.356169,0.0,-1.252050,0.338946,-0.716981,-0.411765,0.802817,-0.181818,0.411765,...,-0.388792,-0.388792,0.411765,0.411765,0.0,-0.5,0.166667,0.166667,0.000,0.000


By using "last" for the newly inserted columns, the NaN values should disappear. However, we need to check if there are patients that have NaN values in their last row.

In [ ]:
# Print the entire row of patients (RecordID) that have NaN values in their last row (last time point)
# Get the last row for each patient (grouped by RecordID)
last_rows = train_set_aug.groupby("RecordID").tail(1)

# Filter to get only those rows with at least one NaN value
missing_last = last_rows[last_rows.isna().any(axis=1)]

# Print the full rows for those patients
print("Patients with NaN values in their last row:")
missing_last

Patients with NaN values in their last row:


,RecordID,Time,Gender,Height,Weight,Age,Albumin,Cholesterol,DiasABP,HCO3,...,Temp_lag1,Temp_lag2,BUN_lag1,BUN_lag2,FiO2_lag1,FiO2_lag2,GCS_lag1,GCS_lag2,K_lag1,K_lag2
19632,133628,2025-03-10 05:00:00,1.0,0.447939,-0.515385,1.356169,0.042792,-0.013487,-2.700621,-0.579257,...,-0.51483,NaN,-0.176471,NaN,2.5,NaN,0.166667,NaN,1.875,NaN
107728,138477,2025-03-10 10:00:00,0.0,-0.704120,-0.141250,-0.194347,-0.105285,-1.196678,0.064058,0.360906,...,-0.51483,NaN,-0.176471,NaN,2.5,NaN,0.166667,NaN,1.875,NaN
117892,139060,2025-03-10 06:00:00,1.0,0.723525,0.775927,-1.285450,0.635100,-1.405476,0.213500,-0.344216,...,-0.51483,NaN,-0.176471,NaN,2.5,NaN,0.166667,NaN,1.875,NaN
144254,140501,2025-03-10 00:00:00,1.0,0.447939,0.351470,-1.917142,1.671639,-0.013487,-0.832594,-0.109176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151735,140936,2025-03-10 00:00:00,1.0,1.327653,0.074041,-0.883465,1.671639,-0.013487,-0.832594,-0.109176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157699,141264,2025-03-10 00:00:00,0.0,-1.526556,-0.800253,1.471022,1.671639,-0.013487,-0.832594,-0.109176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Observe that some patients have NaN values as last row. This happens only for patients with < 3 measurements (hence the lag does not work). We can simply remove them.

In [20]:
# Remove rows that include NaN values
train_aggregated_clean = train_aggregated.dropna()
val_aggregated_clean = val_aggregated.dropna()
test_aggregated_clean = test_aggregated.dropna()
train_aggregated_clean.shape, val_aggregated_clean.shape, test_aggregated_clean.shape


((3994, 62), (3992, 62), (3995, 62))

We didn't loose much, namely 6, 8 and 5 patients.

## Logistic Regression w/ Feature Lagging

In [21]:
def prepare_xy(df):
    X = df.drop(columns=["In-hospital_death", "RecordID"])  # drop non-feature columns
    y = df["In-hospital_death"]  # target column
    return X, y

X_train, y_train = prepare_xy(train_aggregated_clean)
X_valid, y_valid = prepare_xy(val_aggregated_clean)
X_test,  y_test  = prepare_xy(test_aggregated_clean)

# Initialize and train Logistic Regression.
clf = LogisticRegression(random_state=SEED, max_iter=1000)
clf.fit(X_train, y_train)

# Evaluate on validation set.
y_valid_proba = clf.predict_proba(X_valid)[:, 1]
roc_auc_valid = roc_auc_score(y_valid, y_valid_proba)
auprc_valid = average_precision_score(y_valid, y_valid_proba)
print(f"Validation ROC AUC: {roc_auc_valid:.3f}, AUPRC: {auprc_valid:.3f}")

# Evaluate on test set.
y_test_proba = clf.predict_proba(X_test)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Validation ROC AUC: 0.840, AUPRC: 0.492
Test ROC AUC: 0.837, AUPRC: 0.491


## Random Forest w/ Feature Lagging

In [22]:
# Initialize and train Random Forest.
clf = RandomForestClassifier(random_state=SEED)
clf.fit(X_train, y_train)

# Evaluate on validation set.
y_valid_proba = clf.predict_proba(X_valid)[:, 1]
roc_auc_valid = roc_auc_score(y_valid, y_valid_proba)
auprc_valid = average_precision_score(y_valid, y_valid_proba)
print(f"Validation ROC AUC: {roc_auc_valid:.3f}, AUPRC: {auprc_valid:.3f}")

# Evaluate on test set.
y_test_proba = clf.predict_proba(X_test)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Validation ROC AUC: 0.837, AUPRC: 0.487
Test ROC AUC: 0.831, AUPRC: 0.497


***

## Signal processing-based features

Use tsfresh to extract a large amount of features. 

*Note* the use of MinimalFCParameters, to reduce the number of variables inspected (and so the high computational cost) 

Also tryied to substitute w/ EfficientFCParameters but still the run time is too much.

Maybe we should try to use the student cluster but we'll see



In [23]:
from tsfresh import extract_features, select_features
from tsfresh.feature_extraction.settings import MinimalFCParameters
from tsfresh.utilities.dataframe_functions import impute

# Define the column names
id_column = "RecordID"
time_column = "Time"
target_column = "In-hospital_death"

# Step 1: Extract all features from train_set using MinimalFCParameters
X_train_extracted = extract_features(
    train_set.drop(columns=[target_column]),
    column_id=id_column,
    column_sort=time_column,
    default_fc_parameters=MinimalFCParameters()  # Use minimal features
)

# Impute missing values (tsfresh requires no NaNs)
X_train_extracted = impute(X_train_extracted)

# Step 2: Select relevant features based on the target variable
y_train = train_set.groupby(id_column)[target_column].max()
X_train_relevant = select_features(X_train_extracted, y_train)

# Step 3: Store the relevant feature names
relevant_feature_names = X_train_relevant.columns

# Step 4: Extract the same features for val_set
X_val_extracted = extract_features(
    val_set.drop(columns=[target_column]),
    column_id=id_column,
    column_sort=time_column,
    default_fc_parameters=MinimalFCParameters()  # Use the same settings
)
X_val_extracted = impute(X_val_extracted)  # Impute missing values
X_val_relevant = X_val_extracted[relevant_feature_names]  # Keep only relevant features

# Step 5: Extract the same features for test_set
X_test_extracted = extract_features(
    test_set.drop(columns=[target_column]),
    column_id=id_column,
    column_sort=time_column,
    default_fc_parameters=MinimalFCParameters()  # Use the same settings
)
X_test_extracted = impute(X_test_extracted)  # Impute missing values
X_test_relevant = X_test_extracted[relevant_feature_names]  # Keep only relevant features

# Now X_train_relevant, X_val_relevant, and X_test_relevant have the same features
print("Train set relevant features:", X_train_relevant.shape)
print("Validation set relevant features:", X_val_relevant.shape)
print("Test set relevant features:", X_test_relevant.shape)

Feature Extraction: 100%|██████████| 20/20 [00:09<00:00,  2.15it/s]


Train set relevant features: (4000, 230)
Validation set relevant features: (4000, 230)
Test set relevant features: (4000, 230)


In [24]:
# For train_set
y_train = train_set.groupby(id_column)[target_column].max()

# For val_set
y_val = val_set.groupby(id_column)[target_column].max()

# For test_set
y_test = test_set.groupby(id_column)[target_column].max()


### Logistic Regression w/ tsfresh features

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler

# --- Scale the features ---
scaler = StandardScaler()

# Fit the scaler to the training data and transform
X_train_scaled = scaler.fit_transform(X_train_relevant)

# Transform the validation and test sets using the same scaler
X_val_scaled = scaler.transform(X_val_relevant)
X_test_scaled = scaler.transform(X_test_relevant)

# --- Create and train the Logistic Regression classifier ---
clf = LogisticRegression(random_state=42, max_iter=5000)  # Increase iterations to 5000
clf.fit(X_train_scaled, y_train)

# --- Evaluate on the validation set ---
y_val_proba = clf.predict_proba(X_val_scaled)[:, 1]  # Probability for the positive class
roc_auc_val = roc_auc_score(y_val, y_val_proba)
auprc_val = average_precision_score(y_val, y_val_proba)
print(f"Validation ROC AUC: {roc_auc_val:.3f}, AUPRC: {auprc_val:.3f}")

# --- Evaluate on the test set ---
y_test_proba = clf.predict_proba(X_test_scaled)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")


Validation ROC AUC: 0.735, AUPRC: 0.333
Test ROC AUC: 0.797, AUPRC: 0.445


Since there is a total of 215 features exctracted, we'll try with some regularization.

### Logistic Regression w/ tsfresh features && L2 Regularization

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# --- Scale the features ---
scaler = StandardScaler()

# Fit the scaler to the training data and transform
X_train_scaled = scaler.fit_transform(X_train_relevant)

# Transform the validation and test sets using the same scaler
X_val_scaled = scaler.transform(X_val_relevant)
X_test_scaled = scaler.transform(X_test_relevant)

# --- Define the Logistic Regression model ---
# Use L2 regularization (default) and set up the solver
log_reg = LogisticRegression(random_state=42, max_iter=5000, penalty='l2', solver='liblinear')

# --- Set up the grid search for hyperparameter tuning ---
# Define the range of C values to test
param_grid = {'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}

# Use ROC AUC as the scoring metric for grid search
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, scoring='roc_auc', cv=5, verbose=1)

# --- Perform grid search on the training data ---
grid_search.fit(X_train_scaled, y_train)

# --- Get the best model and its parameters ---
best_clf = grid_search.best_estimator_
best_C = grid_search.best_params_['C']
print(f"Best regularization parameter (C): {best_C}")

# --- Evaluate on the validation set using the best model ---
y_val_proba = best_clf.predict_proba(X_val_scaled)[:, 1]  # Probability for the positive class
roc_auc_val = roc_auc_score(y_val, y_val_proba)
auprc_val = average_precision_score(y_val, y_val_proba)
print(f"Validation ROC AUC: {roc_auc_val:.3f}, AUPRC: {auprc_val:.3f}")

# --- Evaluate on the test set using the best model ---
y_test_proba = best_clf.predict_proba(X_test_scaled)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best regularization parameter (C): 0.01
Validation ROC AUC: 0.768, AUPRC: 0.383
Test ROC AUC: 0.813, AUPRC: 0.476


### Logistic Regression w/ tsfresh features && L1 Regularization

Since Lasso convergence is slower, we improved it by using the saga solver, increasing the iterations, adjusting the tolerance and also doing the grid search with every processor (n_jobs=-1)

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# --- Scale the features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_relevant)
X_val_scaled = scaler.transform(X_val_relevant)
X_test_scaled = scaler.transform(X_test_relevant)

# --- Define the Logistic Regression model with L1 regularization ---
log_reg = LogisticRegression(
    penalty='l1', 
    solver='saga', 
    max_iter=5000,  # Increase max_iter
    tol=1e-3,       # Adjust tolerance
    warm_start=True, # Enable warm start
    random_state=42
)
# --- Set up the grid search for hyperparameter tuning ---
param_grid = {'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, scoring='roc_auc', cv=5, n_jobs=-1, verbose=1)

# --- Perform grid search on the training data ---
grid_search.fit(X_train_scaled, y_train)

# --- Get the best model and its parameters ---
best_clf = grid_search.best_estimator_
best_C = grid_search.best_params_['C']
print(f"Best regularization parameter (C): {best_C}")

# --- Evaluate on the validation set using the best model ---
y_val_proba = best_clf.predict_proba(X_val_scaled)[:, 1]
roc_auc_val = roc_auc_score(y_val, y_val_proba)
auprc_val = average_precision_score(y_val, y_val_proba)
print(f"Validation ROC AUC: {roc_auc_val:.3f}, AUPRC: {auprc_val:.3f}")

# --- Evaluate on the test set using the best model ---
y_test_proba = best_clf.predict_proba(X_test_scaled)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best regularization parameter (C): 0.1
Validation ROC AUC: 0.777, AUPRC: 0.397
Test ROC AUC: 0.816, AUPRC: 0.479


### Random Forest w/ tsfresh features

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score, average_precision_score
import numpy as np

# --- Define the Random Forest Classifier ---
rf = RandomForestClassifier(random_state=42, n_jobs=-1)  # Use all CPU cores

# --- Set up the randomized search for hyperparameter tuning ---
param_dist = {
    'n_estimators': [100, 200],  # Number of trees
    'max_depth': [10, 20],       # Maximum depth of each tree
    'min_samples_split': [2, 5], # Minimum samples to split a node
    'min_samples_leaf': [1, 2],  # Minimum samples at each leaf node
    'max_features': ['sqrt', 'log2']  # Number of features to consider at each split
}

# Use RandomizedSearchCV with a fixed number of iterations
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=10,  # Number of hyperparameter combinations to try
    scoring='roc_auc',
    cv=5,
    n_jobs=-1,  # Use all CPU cores
    verbose=1,
    random_state=42
)

# --- Perform randomized search on the training data ---
random_search.fit(X_train_relevant, y_train)

# --- Get the best model and its parameters ---
best_clf = random_search.best_estimator_
best_params = random_search.best_params_
print(f"Best hyperparameters: {best_params}")

# --- Evaluate on the validation set using the best model ---
y_val_proba = best_clf.predict_proba(X_val_relevant)[:, 1]  # Probability for the positive class
roc_auc_val = roc_auc_score(y_val, y_val_proba)
auprc_val = average_precision_score(y_val, y_val_proba)
print(f"Validation ROC AUC: {roc_auc_val:.3f}, AUPRC: {auprc_val:.3f}")

# --- Evaluate on the test set using the best model ---
y_test_proba = best_clf.predict_proba(X_test_relevant)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best hyperparameters: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 20}
Validation ROC AUC: 0.820, AUPRC: 0.452
Test ROC AUC: 0.810, AUPRC: 0.465


Without hyperparameter tuning:

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score

# --- Definisci il modello Random Forest con parametri predefiniti ---
rf = RandomForestClassifier(random_state=42, n_jobs=-1)  # Usa tutti i core della CPU

# --- Addestra il modello sui dati di training ---
rf.fit(X_train_relevant, y_train)

# --- Valuta sul validation set ---
y_val_proba = rf.predict_proba(X_val_relevant)[:, 1]  # Probabilità per la classe positiva
roc_auc_val = roc_auc_score(y_val, y_val_proba)
auprc_val = average_precision_score(y_val, y_val_proba)
print(f"Validation ROC AUC: {roc_auc_val:.3f}, AUPRC: {auprc_val:.3f}")

# --- Valuta sul test set ---
y_test_proba = rf.predict_proba(X_test_relevant)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Validation ROC AUC: 0.806, AUPRC: 0.421
Test ROC AUC: 0.778, AUPRC: 0.422


***

# Q2.2 - Recurrent Neural Networks

Working with LSTM, we figured out it was better scaling the data using only MinMaxScaler. Let's do it then!

In [17]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from project_1.config import PROJ_ROOT, PROCESSED_DATA_DIR
from project_1.loading import *
from project_1.dataset import *
from project_1.features import *

set_a, set_b, set_c = load_before_scaling()
# Scale using basic scaling
set_a_scaled, set_b_scaled, set_c_scaled = scale_features_basic(set_a, set_b, set_c)

# Remove the ICUType feature
set_a_scaled = set_a_scaled.drop(columns=['ICUType'])
set_b_scaled = set_b_scaled.drop(columns=['ICUType'])
set_c_scaled = set_c_scaled.drop(columns=['ICUType'])
death_a, death_b, death_c = load_outcomes()

Shapes of the datasets:
Set A: (183416, 43) Set B: (183495, 43) Set C: (183711, 43)
Shapes of labels:
Set A: (4000, 2) Set B: (4000, 2) Set C: (4000, 2)


## LSTM (Long Short Term Memory) w/ Taking last Hidden State

In [18]:
import torch
import torch.nn as nn

class LSTM_Model(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, num_classes=1, dropout=0.3):
        super(LSTM_Model, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,       # 41 features per time step
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        out, _ = self.lstm(x)           # out: (batch_size, seq_len, hidden_size)
        out = out[:, -1, :]             # Take last time step: (batch_size, hidden_size)
        out = self.fc(out)              # (batch_size, num_classes)
        return out.squeeze()            # (batch_size,) for BCEWithLogitsLoss

### Create TensorDatasets from Time-Series data

In [19]:
train_dataset = create_dataset_from_timeseries(set_a_scaled, death_a["In-hospital_death"])
validation_dataset = create_dataset_from_timeseries(set_b_scaled, death_b["In-hospital_death"])
test_dataset = create_dataset_from_timeseries(set_c_scaled, death_c["In-hospital_death"])

train_dataset.tensors[0].shape # (batch_size, seq_len, input_size)

torch.Size([4000, 49, 40])

In [4]:
# Convert to DataLoader
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Train LSTM

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = train_dataset.tensors[0].shape[-1]
model = LSTM_Model(input_size=input_size).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Call the trainig loop (default 10 epochs)
model = train_model_with_validation(model, train_loader, validation_loader, criterion, optimizer, device)

/Users/francescobondi/anaconda3/envs/TUM/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1/20
  Train Loss: 0.4519 | AUCROC: 0.5865 | AUPRC: 0.1663
  Val   Loss: 0.3444 | AUCROC: 0.7785 | AUPRC: 0.4238



Epoch 11/20
  Train Loss: 0.1661 | AUCROC: 0.9586 | AUPRC: 0.8283
  Val   Loss: 0.4117 | AUCROC: 0.7958 | AUPRC: 0.3913



# Evaluation LSTM

In [6]:
avg_loss, aucroc, auprc = evaluate_model(model, test_loader, criterion, device)
print(f"Test Loss: {avg_loss:.4f}, AUC-ROC: {aucroc:.4f}, AUC-PRC: {auprc:.4f}")

Evaluation - Loss: 0.6900 - AUCROC: 0.7586 - AUPRC: 0.3453
Test Loss: 0.6900, AUC-ROC: 0.7586, AUC-PRC: 0.3453


## LSTM w/ Mean Pooling

In [7]:
class LSTM_Model_Pooling(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, num_classes=1, dropout=0.3):
        super(LSTM_Model_Pooling, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,       # 40 features per time step
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        out, _ = self.lstm(x)           # out: (batch_size, seq_len, hidden_size)
        out = out.mean(dim=1)           # Pooling: (batch_size, hidden_size)   
        out = self.fc(out)              # (batch_size, num_classes)
        return out.squeeze()            # (batch_size,) for BCEWithLogitsLoss

In [8]:
# Use the previous data loaders and train the new model
model_pooling = LSTM_Model_Pooling(input_size=input_size).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_pooling.parameters(), lr=0.001)

model_pooling = train_model_with_validation(model_pooling, train_loader, validation_loader, criterion, optimizer, device)

Epoch 1/20
  Train Loss: 0.4577 | AUCROC: 0.5412 | AUPRC: 0.1512
  Val   Loss: 0.3753 | AUCROC: 0.7340 | AUPRC: 0.3125



Epoch 11/20
  Train Loss: 0.1942 | AUCROC: 0.9426 | AUPRC: 0.7749
  Val   Loss: 0.4011 | AUCROC: 0.7944 | AUPRC: 0.3882



In [9]:
# Now evaluate the model
avg_loss, aucroc, auprc = evaluate_model(model_pooling, test_loader, criterion, device)
print(f"Test Loss: {avg_loss:.4f}, AUC-ROC: {aucroc:.4f}, AUC-PRC: {auprc:.4f}")

Evaluation - Loss: 0.7406 - AUCROC: 0.7532 - AUPRC: 0.3628
Test Loss: 0.7406, AUC-ROC: 0.7532, AUC-PRC: 0.3628


## LSTM w/ Max Pooling

In [10]:
class LSTM_Model_Max_Pooling(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, num_classes=1, dropout=0.3):
        super(LSTM_Model_Max_Pooling, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,       # 40 features per time step
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        out, _ = self.lstm(x)           # out: (batch_size, seq_len, hidden_size)
        out, _ = out.max(dim=1)           # Pooling: (batch_size, hidden_size)   
        out = self.fc(out)              # (batch_size, num_classes)
        return out.squeeze()            # (batch_size,) for BCEWithLogitsLoss

In [11]:
# Use the previous data loaders and train the new model
model_max_pooling = LSTM_Model_Max_Pooling(input_size=input_size).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_max_pooling.parameters(), lr=0.001)

model_max_pooling = train_model_with_validation(model_max_pooling, train_loader, validation_loader, criterion, optimizer, device)

Epoch 1/20
  Train Loss: 0.4482 | AUCROC: 0.5428 | AUPRC: 0.1499
  Val   Loss: 0.3999 | AUCROC: 0.7142 | AUPRC: 0.3002



Epoch 11/20
  Train Loss: 0.1960 | AUCROC: 0.9376 | AUPRC: 0.7655
  Val   Loss: 0.4062 | AUCROC: 0.8076 | AUPRC: 0.4017



In [13]:
# Now evaluate the model
avg_loss, aucroc, auprc = evaluate_model(model_max_pooling, test_loader, criterion, device)
print(f"Test Loss: {avg_loss:.4f}, AUC-ROC: {aucroc:.4f}, AUC-PRC: {auprc:.4f}")

Evaluation - Loss: 0.5317 - AUCROC: 0.7786 - AUPRC: 0.3793
Test Loss: 0.5317, AUC-ROC: 0.7786, AUC-PRC: 0.3793


## Bidirectional LSTM

In [14]:
class LSTM_Model_Bi(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, num_classes=1, dropout=0.3):
        super(LSTM_Model_Bi, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,       # 41 features per time step
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True
        )
        self.fc = nn.Linear(hidden_size * 2, num_classes) # *2 for bidirectional

    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        out, _ = self.lstm(x)           # out: (batch_size, seq_len, hidden_size)
        out = out[:, -1, :]             # Take last time step: (batch_size, hidden_size)
        out = self.fc(out)              # (batch_size, num_classes)
        return out.squeeze()            # (batch_size,) for BCEWithLogitsLoss

In [15]:
# Train the model
model_bi = LSTM_Model_Bi(input_size=input_size).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_bi.parameters(), lr=0.001)

model_bi = train_model_with_validation(model_bi, train_loader, validation_loader, criterion, optimizer, device)

Epoch 1/20
  Train Loss: 0.4522 | AUCROC: 0.5534 | AUPRC: 0.1687
  Val   Loss: 0.3486 | AUCROC: 0.7666 | AUPRC: 0.3994



Epoch 11/20
  Train Loss: 0.1743 | AUCROC: 0.9523 | AUPRC: 0.8123
  Val   Loss: 0.3777 | AUCROC: 0.8174 | AUPRC: 0.4125



In [ ]:
# Now evaluate
avg_loss, aucroc, auprc = evaluate_model(model_bi, test_loader, criterion, device)
print(f"Test Loss: {avg_loss:.4f}, AUC-ROC: {aucroc:.4f}, AUC-PRC: {auprc:.4f}")

***

# Q2.3a: Transformers

## Transformer w/ Positional Encoding

In [ ]:
import torch.nn

class TransformerClassifier(nn.Module):
    def __init__(self, input_size, num_classes=1, nhead=4, num_layers=2, dim_feedforward=128, dropout=0.3):
        super().__init__()
        self.input_size = input_size

        # Project input features to model dimension
        self.embedding = nn.Linear(input_size, dim_feedforward)

        # Positional Encoding
        self.pos_encoder = PositionalEncoding(dim_feedforward, dropout)

        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=dim_feedforward,
            nhead=nhead,
            dim_feedforward=dim_feedforward * 2,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Final classifier
        self.fc = nn.Linear(dim_feedforward, num_classes)

    def forward(self, x):
        # x: (batch, seq_len, input_size)

        if x.dim() == 2:
            x = x.unsqueeze(1) # (batch, 1, input_size)

        x = self.embedding(x)                # (batch, seq_len, d_model)
        #print("After embedding:", x.shape)  # Debug print
        x = self.pos_encoder(x)
        #print("After pos encoding:", x.shape)  # Debug print
        x = self.transformer_encoder(x)      # (batch, seq_len, d_model)
        #print("After transformer encoder:", x.shape)

        x = x.mean(dim=1)                    # mean pooling over time
        #print("After pooling:", x.shape)     # Debug print
        out = self.fc(x).squeeze()           # (batch,)
        #print("After fc:", out.shape)        # Debug print
        return out


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=500):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(torch.log(torch.tensor(10000.0)) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

In [ ]:
# Train the Transformer model
model_transformer = TransformerClassifier(input_size=input_size).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_transformer.parameters(), lr=0.001)

model_transformer = train_model_with_validation(model_transformer, train_loader, validation_loader, criterion, optimizer, device)

In [ ]:
# Evaluate the model
avg_loss, aucroc, auprc = evaluate_model(model_transformer, test_loader, criterion, device)
print(f"Test Loss: {avg_loss:.4f}, AUC-ROC: {aucroc:.4f}, AUC-PRC: {auprc:.4f}")

***

# Q2.3b: Tokenizing Time-series Data and Transformers

In [ ]:
# For this part, we need to load the initial data
set_a_initial, set_b_initial, set_c_initial = load_basic_data()
set_a_initial.head()

## Create the TZV Dataframe (following Horn et al.)

In [16]:
from sklearn.preprocessing import MinMaxScaler

def build_TZV_dataframe(original_df, label_df, base_time="2025-03-10 00:00:00", duration_hours=48):
    """
    Build a long-format dataframe with columns [T, Z, V, y] from an original wide dataframe.
    
    Parameters:
        original_df (pd.DataFrame): DataFrame with columns [RecordID, Time, f1, f2, ..., f41].
        label_df (pd.DataFrame): DataFrame with columns [RecordID, y] containing the label for each RecordID.
        base_time (str): Base time used for normalizing the Time column.
        duration_hours (int): The duration (in hours) from base_time over which Time is normalized (here, 48 hours).
    
    Returns:
        long_df (pd.DataFrame): Long-format dataframe with columns:
                                T: normalized time [0, 1],
                                Z: index of the feature,
                                V: scaled measurement value,
                                y: label corresponding to RecordID.
        feature_to_index (dict): Mapping from original feature names to integer indices.
    """
    # Merge the labels with the original dataframe using RecordID.
    df = original_df.copy().merge(label_df, on="RecordID", how="left")
    
    # Convert Time to datetime and compute normalized time T.
    df["Time"] = pd.to_datetime(df["Time"])
    start_time = pd.to_datetime(base_time)
    end_time = start_time + pd.Timedelta(hours=duration_hours)
    total_seconds = (end_time - start_time).total_seconds()
    df["T"] = (df["Time"] - start_time).dt.total_seconds() / total_seconds
    
    # Identify feature columns: all columns except RecordID, Time, T, and y.
    feature_cols = [col for col in df.columns if col not in ["RecordID", "Time", "T", "In-hospital_death"]]
    
    # Scale each feature individually using MinMaxScaler.
    scaler = MinMaxScaler()
    df[feature_cols] = scaler.fit_transform(df[feature_cols])
    
    # Melt the dataframe from wide to long format.
    # The id_vars ("T" and "y") are preserved for each measurement.
    long_df = pd.melt(df, id_vars=["T", "In-hospital_death"], value_vars=feature_cols, 
                      var_name="Z", value_name="V")
    
    # Map feature names to indices for the "Z" column.
    feature_to_index = {feat: idx for idx, feat in enumerate(feature_cols)}
    long_df["Z"] = long_df["Z"].map(feature_to_index)
    
    # Sort the final dataframe by normalized time T and reset the index.
    long_df = long_df.sort_values("T").reset_index(drop=True)
    long_df = long_df.dropna(subset=["V"])
    
    return long_df, feature_to_index

In [ ]:
# Build the TZV dataframes
TZV_a, feature_to_index_a = build_TZV_dataframe(set_a_initial, death_a)
TZV_b, feature_to_index_b = build_TZV_dataframe(set_b_initial, death_b)
TZV_c, feature_to_index_c = build_TZV_dataframe(set_c_initial, death_c)

print(TZV_a.shape)
TZV_a.head(10)

In [ ]:
# Check for the total number of not NaN values under some specified columns
selected_cols = [col for col in set_a_initial.columns if col not in ["RecordID", "Time"]]
set_a_initial[selected_cols].notna().sum().sum()

Checked that the number of not NaN values is the same as the rows of the new dataframe! Let's go!

## Train the TZV Format with a Transformer

In [ ]:
# Remove the In-hospital_death column from the TZV dataframes, but save it
y_a = TZV_a.pop("In-hospital_death")
y_b = TZV_b.pop("In-hospital_death")
y_c = TZV_c.pop("In-hospital_death")

# Convert the TZV dataframes to PyTorch tensors
X_a = torch.tensor(TZV_a[["T", "Z", "V"]].values, dtype=torch.float32)
X_b = torch.tensor(TZV_b[["T", "Z", "V"]].values, dtype=torch.float32)
X_c = torch.tensor(TZV_c[["T", "Z", "V"]].values, dtype=torch.float32)
print(X_a.shape, X_b.shape, X_c.shape)

# Create the datasets and dataloaders
from torch.utils.data import TensorDataset

dataset_a = TensorDataset(X_a, torch.tensor(y_a.values, dtype=torch.float32))
dataset_b = TensorDataset(X_b, torch.tensor(y_b.values, dtype=torch.float32))
dataset_c = TensorDataset(X_c, torch.tensor(y_c.values, dtype=torch.float32))

# Use pin_memory and increase the num_workers for faster data loading on GPU
loader_a = DataLoader(dataset_a, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
loader_b = DataLoader(dataset_b, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
loader_c = DataLoader(dataset_c, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

### Training Loop (beware, the size of the DFs makes it hard to run on simple laptops. We used the ETH Cluster to run it.)

In [ ]:
model_tvz = TransformerClassifier(input_size=3).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_tvz.parameters(), lr=0.001)

model_tvz = train_model_with_validation(model_tvz, loader_a, loader_b, criterion, optimizer, device)

In [ ]:
# Evaluate the model
avg_loss, aucroc, auprc = evaluate_model(model_tvz, loader_c, criterion, device)
print(f"Test Loss: {avg_loss:.4f}, AUC-ROC: {aucroc:.4f}, AUC-PRC: {auprc:.4f}")